In [ ]:
!pip install tensorflow

!pip install --upgrade keras

!pip install --upgrade matplotlib

import matplotlib.pyplot as plt

import pickle
import numpy as np

In [ ]:
import pandas as pd

In [1]:
import os
import pickle

# Directory where the extracted bAbI files are located
input_directory = r"C:\Onedrive IITK\OneDrive - IIT Kanpur\Project Data science\My chatbot\tasks_1-20_v1-2\en-10k"  # Update this path accordingly

# Specify the file names
train_file_name = "qa6_yes-no-questions_train.txt"
test_file_name = "qa6_yes-no-questions_test.txt"

# Construct full file paths
train_file_path = os.path.join(input_directory, train_file_name)
test_file_path = os.path.join(input_directory, test_file_name)

# Read train and test files
with open(train_file_path, 'r', encoding='utf-8') as train_file:
    train_data = train_file.read()

with open(test_file_path, 'r', encoding='utf-8') as test_file:
    test_data = test_file.read()

# Save as pickle files
train_pickle_file = "train_data.pickle"
test_pickle_file = "test_data.pickle"

with open(train_pickle_file, 'wb') as train_pickle:
    pickle.dump(train_data, train_pickle)

with open(test_pickle_file, 'wb') as test_pickle:
    pickle.dump(test_data, test_pickle)

print(f"Train data saved as {train_pickle_file}")
print(f"Test data saved as {test_pickle_file}")

Train data saved as train_data.pickle
Test data saved as test_data.pickle


In [ ]:
train_data.pickle


In [ ]:
train_pickle

In [ ]:
train_data=pickle.load(train_pickle)

In [ ]:
with open("train_data.pickle","rb") as fp:
    train_data=pickle.load(fp)

In [ ]:
with open("test_data.pickle","rb") as fp:
    test_data=pickle.load(fp)

In [ ]:
type(test_data)

In [ ]:
len(test_data)

In [ ]:
test_data

In [ ]:
def parse_string_to_list(data_string):
    data_list = []
    lines = data_string.strip().split('\n')
    for i in range(2, len(lines)):
        current_line = lines[i]
        parts = current_line.split('\t')
        if len(parts) >= 2:
            #question = parts[0].strip().split()
            question = [word for word in parts[0].strip().split() if not word.isdigit()]
            answer = parts[1].strip()
              # Extract previous two lines and split them word-wise into lists
            prev1_words = [word for word in lines[i-1].strip().split() if not word.isdigit()]
            prev2_words = [word for word in lines[i-2].strip().split() if not word.isdigit()]
            
            # Join the first two sentences into a single string
            prev1_and_prev2 = ' '.join(prev2_words + prev1_words)
            
            # Append a tuple of combined previous two lines, question, and answer to data_list
            data_list.append((prev1_and_prev2.split(), question, answer))
    return data_list
# Parse string data into list of tuples
test_data = parse_string_to_list(test_data)

# Print the parsed data
print(test_data)

In [ ]:
train_data


In [ ]:
train_data

In [ ]:
def parse_string_to_list(data_string):
    data_list = []
    lines = data_string.strip().split('\n')
    for i in range(2, len(lines)):
        current_line = lines[i]
        parts = current_line.split('\t')
        if len(parts) >= 2:
            #question = parts[0].strip().split()
            question = [word for word in parts[0].strip().split() if not word.isdigit()]
            answer = parts[1].strip()
              # Extract previous two lines and split them word-wise into lists
            prev1_words = [word for word in lines[i-1].strip().split() if not word.isdigit()]
            prev2_words = [word for word in lines[i-2].strip().split() if not word.isdigit()]
            
            # Join the first two sentences into a single string
            prev1_and_prev2 = ' '.join(prev2_words + prev1_words)
            
            # Append a tuple of combined previous two lines, question, and answer to data_list
            data_list.append((prev1_and_prev2.split(), question, answer))
    return data_list

# Parse string data into list of tuples
train_data = parse_string_to_list(train_data)

# Print the parsed data
print(train_data)



In [ ]:
train_data

In [ ]:
train_data[0][2]


In [ ]:
type(test_data)

In [ ]:
len(train_data)

In [ ]:
' '.join(train_data[0][0])

In [ ]:
' '.join(train_data[0][1])

In [ ]:
train_data[0][2]

In [ ]:
vocab = set()

In [ ]:
all_data = test_data+train_data

In [ ]:
for story, question, answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [ ]:
vocab.add('yes')
vocab.add('no')

In [ ]:
vocab

In [ ]:
all_data

In [ ]:
len (vocab)

In [ ]:
vocab_len=len(vocab)+1

In [ ]:
for data in all_data:
    print (len(data[0]))
    print('\n')

In [ ]:
max_story_len = max([len(data[0]) for data in all_data])
max_ques_len = max([len(data[1]) for data in all_data])

In [ ]:
vocab

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer



In [ ]:
tokenizer = Tokenizer(filters=[])

In [ ]:
tokenizer.fit_on_texts(vocab)

In [ ]:
tokenizer.word_index

In [ ]:
train_story_text = []
train_question_text = []
train_answers = []

for story, question, answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)

In [ ]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [ ]:
 len(train_story_text)

In [ ]:
train_story_seq

In [ ]:
def vectorize_stories(data, word_index = tokenizer.word_index, max_story_len = max_story_len, max_ques_len = max_ques_len):
    X=[]#stories
    Xq=[] #query/question
    Y=[] #correct answer
    for story, query, answer in data:
        x=[word_index[word.lower()] for word in story]
        xq=[word_index[word.lower()] for word in query]
        y= np.zeros(len(word_index)+1)
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return(pad_sequences(X, maxlen = max_story_len),pad_sequences(X, maxlen = max_ques_len),np.array(Y))

In [ ]:
inputs_train,queries_train,answers_train = vectorize_stories(train_data)

In [ ]:
inputs_test,queries_test,answers_test = vectorize_stories(test_data)

In [ ]:
inputs_train

In [ ]:
queries_train

In [ ]:
answers_test

In [ ]:
answers_train

In [ ]:
tokenizer.word_index['yes']

In [ ]:
tokenizer.word_index['no']

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM

In [ ]:
input_sequence = Input((max_story_len,))

In [ ]:
question = Input((max_ques_len,))

In [ ]:
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim = vocab_len, output_dim = 64))
input_encoder_m.add(Dropout(0.3))

In [ ]:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim = vocab_len, output_dim = max_ques_len))
input_encoder_c.add(Dropout(0.3))

In [ ]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim = vocab_len, output_dim = 64, input_length = max_ques_len))
question_encoder.add(Dropout(0.3))

In [ ]:
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [ ]:
match = dot([input_encoded_m, question_encoded], axes=(2,2))
match = Activation('softmax')(match)

In [ ]:
response = add([match,input_encoded_c])
response = Permute((2,1))(response)

In [ ]:
answer = concatenate([response,question_encoded])

In [ ]:
answer

In [ ]:
answer = LSTM(32)(answer)

In [ ]:
answer = Dropout(0.5)(answer)
answer = Dense(vocab_len)(answer)

In [ ]:
answer = Activation('softmax')(answer)


In [ ]:
model = Model([input_sequence, question], answer)

In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit([inputs_train, queries_train], answers_train, batch_size = 32, epochs = 20, validation_data = ([inputs_test, queries_test], answers_test))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("epochs")
plt.legend(["train", "test"])

In [ ]:
model.save("chatbot_model.keras")

In [ ]:
model.load_weights("chatbot_model.keras")

In [ ]:
pred_results = model.predict(([inputs_test, queries_test]))

In [ ]:

test_data[0][0]

In [ ]:
story = ' '.join(word for word in test_data[10][0])

In [ ]:
story


In [ ]:
query = ' '.join(word for word in test_data[10][1])

In [ ]:
query

In [ ]:
test_data[10][2]

In [ ]:
val_max=np.argmax(pred_results[10])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k=key
        
print("Predicted Answer is", k)
print("Probability of certainty",pred_results[10][val_max])

In [ ]:
vocab


In [ ]:
#make our own story

In [ ]:
story = "Mary dropped the football. Sandra discarded the apple in kitchen. Daniel went to office." 

In [ ]:
story.split()

In [ ]:
my_question = "Is Daniel in the office."
my_question.split()

In [ ]:
mydata = [(story.split(), my_question.split(),'yes')]

In [ ]:
my_story, my_ques, my_ans = vectorize_stories(mydata)

In [ ]:
pred_results = model.predict(([my_story,my_ques]))

In [ ]:
val_max=np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k=key
        
print("Predicted Answer is", k)
print("Probability of certainty",pred_results[0][val_max])